## Cong's Backtesting Python Script Mar 2022 - Verticals - Pandas rewrite - Working

This code backtests the performance of an at-the-money SPY put (sold) and a SPY put (bought) 25 points below, opened every Wednesday from Jan 2022 to May 2022. 

- This file reads from various data sources, TDA Ameritrade (in_tda), CBOE (in_cboe), Yahoo Finance (in_yahoo), combines them into a new df (in_all) and outputs a df called out.
- The backtesting involves running the same trade through many different combinations of variables (DTE, days_to_close, take_prof, take_loss) to find out what was the ideal parameter set, using grid search.
- A total of 7,200 possible combinations were evaluated.
- Once this ideal set of parameters are determined, trades can be placed while monitoring and comparing the performance of future trades to past trades.
- Please contact chuayaocong@gmail.com if you require assistance. Do note that the source files will not be provided due to the large data size (> 3GB).

### Metadata:

In [4]:
'''
- DTE            - days to expiry (counting from open_date, will offset slightly (a few days) depending on when is expiry Friday)
- days_to_close  - no. of days before closing trade, assuming no early exit
- take_prof      - a ratio of credit received. once this ratio is exceeded, to take profit   
- take_loss      - a ratio of max loss. once this ratio is exceeded, to take profit  
- spy_upper      - upper leg of vertical (assume bullish vertical)
- spy_lower      - lower leg of vertical (assume bullish vertical)
- spy_upp+0      - the price of upper leg on Day 0
- spy_lower+0    - the price of lower leg on Day 0
- credit_0       - the credit received on Day 0 for opening the trade
- norm_pl+7      - the P/L if the trade is closed normally, i.e. no early exit
- norm_pl        - the consolidated PL
- takeprof_day+5 - boolean. if True, this means take profit is triggered on Day 5
- takeloss_day+5 - boolean. if True, this means take loss is triggered on Day 5
- credit_day     - an intermediate value computed for credit value of options on a particular day (not impt)
- early_prof+7   - P/L if early profit is taken on Day 7
- early_loss+7   - P/L if early loss is taken on Day 7 (+ve value means gain, -ve value means loss)
- final_pl       - final P/L of a particular trade taking into account normal exit and early exit
- max_loss       - theoretical max loss
- final_pl%      - final P/L% of a particular trade
- combi_no       - serial no each denoting a set of trades with same params (DTE, days to close, take prof, take loss)
- mean_pl%       - computes the mean P/L of each set of params
'''

'\n- DTE            - days to expiry (counting from open_date, will offset slightly (a few days) depending on when is expiry Friday)\n- days_to_close  - no. of days before closing trade, assuming no early exit\n- take_prof      - a ratio of credit received. once this ratio is exceeded, to take profit   \n- take_loss      - a ratio of max loss. once this ratio is exceeded, to take profit  \n- spy_upper      - upper leg of vertical (assume bullish vertical)\n- spy_lower      - lower leg of vertical (assume bullish vertical)\n- spy_upp+0      - the price of upper leg on Day 0\n- spy_lower+0    - the price of lower leg on Day 0\n- credit_0       - the credit received on Day 0 for opening the trade\n- norm_pl+7      - the P/L if the trade is closed normally, i.e. no early exit\n- norm_pl        - the consolidated PL\n- takeprof_day+5 - boolean. if True, this means take profit is triggered on Day 5\n- takeloss_day+5 - boolean. if True, this means take loss is triggered on Day 5\n- credit_day

## 1a. Import from TDA

In [5]:
import pandas as pd            #opens all CSV files and adds them to dataframe
import glob
import os
from pprint import pprint
import pandas_datareader.data as web
import numpy as np
import datetime
from time import time
import math

pd.set_option('display.max_columns', None)

In [6]:
print("Reading csv ...")
beforetime = time()

in_tda = pd.read_csv("C:\\Users\\cai_t\\Desktop\\2022 files\\_BACKTEST\\rstudio-export\\combined.csv", index_col=0)

in_tda = in_tda[["putCall","symbol","description","mark","inTheMoney",
               "quoteTimeInLong","volatility","delta","gamma",
               "theta","vega","rho","strikePrice","expirationDate",
               "daysToExpiration"]]

quoteTimeInLong_series = (pd.to_datetime(in_tda["quoteTimeInLong"])).copy() #creates new column quotetimeinlong in timestamp format
quoteTimeInLong_series = quoteTimeInLong_series.rename("quoteTimeInLong_ts")    #renames column to _ts
in_tda = pd.concat([in_tda,quoteTimeInLong_series], sort=False, axis=1)
display(in_tda)
print("time taken to read csv: ",time()-beforetime)

Reading csv ...


C:\Users\cai_t\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,putCall,symbol,description,mark,inTheMoney,quoteTimeInLong,volatility,delta,gamma,theta,vega,rho,strikePrice,expirationDate,daysToExpiration,quoteTimeInLong_ts
Unnamed: 0,,,,,,,,,,,,,,,,
1,CALL,SPY_021622C285,SPY Feb 16 2022 285 Call (Weekly),0.00,True,2022-02-16 21:14:22,0.000,0.638,0.0,188.996,0.0,0.005,285.0,2022-02-16 21:00:00,0.0,2022-02-16 21:14:22
2,CALL,SPY_021622C290,SPY Feb 16 2022 290 Call (Weekly),0.00,True,2022-02-16 21:14:22,0.000,0.000,0.0,0.000,0.0,0.000,290.0,2022-02-16 21:00:00,0.0,2022-02-16 21:14:22
3,CALL,SPY_021622C295,SPY Feb 16 2022 295 Call (Weekly),0.00,True,2022-02-16 21:14:44,0.000,0.000,0.0,0.000,0.0,0.000,295.0,2022-02-16 21:00:00,0.0,2022-02-16 21:14:44
4,CALL,SPY_021622C300,SPY Feb 16 2022 300 Call (Weekly),0.00,True,2022-02-16 21:14:44,0.000,0.000,0.0,0.000,0.0,0.000,300.0,2022-02-16 21:00:00,0.0,2022-02-16 21:14:44
5,CALL,SPY_021622C305,SPY Feb 16 2022 305 Call (Weekly),0.00,True,2022-02-16 21:14:44,0.000,0.000,0.0,0.000,0.0,0.000,305.0,2022-02-16 21:00:00,0.0,2022-02-16 21:14:44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9520,PUT,SPY_122024P700,SPY Dec 20 2024 700 Put,285.44,True,2022-05-27 20:14:40,17.349,-1.000,0.0,0.000,0.0,0.000,700.0,2024-12-20 21:00:00,936.0,2022-05-27 20:14:40
9521,PUT,SPY_122024P705,SPY Dec 20 2024 705 Put,290.44,True,2022-05-27 20:14:40,17.355,-1.000,0.0,0.000,0.0,0.000,705.0,2024-12-20 21:00:00,936.0,2022-05-27 20:14:40
9522,PUT,SPY_122024P710,SPY Dec 20 2024 710 Put,295.43,True,2022-05-27 20:14:40,17.350,-1.000,0.0,0.000,0.0,0.000,710.0,2024-12-20 21:00:00,936.0,2022-05-27 20:14:40


time taken to read csv:  85.31009697914124


## 1b. Import CSVs from CBOE datashop (local drive)

In [7]:
files = os.path.join("C:\\Users\\cai_t\\Desktop\\2022 files\\_BACKTEST\\rstudio-export\\CBOE purchase", "UnderlyingOptionsEOD*.csv")
files = glob.glob(files)
print("Joining files ... ");
in_cboe = pd.concat(map(pd.read_csv, files), ignore_index=True)
# pprint(df)

in_cboe = in_cboe[["option_type","underlying_symbol","quote_date","expiration","strike","open",
                  "high","low","close","underlying_bid_eod"]]

quote_date = (pd.to_datetime(in_cboe["quote_date"])).copy() #creates new column quotetimeinlong in timestamp format
quote_date = quote_date.rename("quoteTimeInLong_ts")    #renames column to _ts
in_cboe = pd.concat([in_cboe,quote_date], sort=False, axis=1)

lista=[]

for i in range(len(in_cboe['expiration'])):               #process text string to format SPY_021622C285 for e.g.
    lista.append(in_cboe['underlying_symbol'][i] + "_" +
                 in_cboe['expiration'][i][5:7] + 
                 in_cboe['expiration'][i][8:10] +
                 in_cboe['expiration'][i][2:4] +
                 in_cboe['option_type'][i] +
                 str(int(in_cboe['strike'][i])))

in_cboe['symbol'] = lista                                   #creates and stores in new column 'symbol' in df
in_cboe['mark'] = in_cboe['close']
in_all0 = pd.concat([in_cboe, in_tda])                  #combine 2 datasets into 1

display(in_all0)
# print(in_all['symbol'])

Joining files ... 


,option_type,underlying_symbol,quote_date,expiration,strike,open,high,low,close,underlying_bid_eod,quoteTimeInLong_ts,symbol,mark,putCall,description,inTheMoney,quoteTimeInLong,volatility,delta,gamma,theta,vega,rho,strikePrice,expirationDate,daysToExpiration
0,C,SPY,2022-01-03,2022-09-16,140.0,0.0,0.0,0.0,0.0,477.59,2022-01-03 00:00:00,SPY_091622C140,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P,SPY,2022-01-03,2022-09-16,140.0,0.0,0.0,0.0,0.0,477.59,2022-01-03 00:00:00,SPY_091622P140,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C,SPY,2022-01-03,2022-09-16,145.0,0.0,0.0,0.0,0.0,477.59,2022-01-03 00:00:00,SPY_091622C145,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P,SPY,2022-01-03,2022-09-16,145.0,0.0,0.0,0.0,0.0,477.59,2022-01-03 00:00:00,SPY_091622P145,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C,SPY,2022-01-03,2022-09-16,150.0,0.0,0.0,0.0,0.0,477.59,2022-01-03 00:00:00,SPY_091622C150,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-27 20:14:40,SPY_122024P700,285.44,PUT,SPY Dec 20 2024 700 Put,True,2022-05-27 20:14:40,17.349,-1.0,0.0,0.0,0.0,0.0,700.0,2024-12-20 21:00:00,936.0
9521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-27 20:14:40,SPY_122024P705,290.44,PUT,SPY Dec 20 2024 705 Put,True,2022-05-27 20:14:40,17.355,-1.0,0.0,0.0,0.0,0.0,705.0,2024-12-20 21:00:00,936.0
9522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-27 20:14:40,SPY_122024P710,295.43,PUT,SPY Dec 20 2024 710 Put,True,2022-05-27 20:14:40,17.350,-1.0,0.0,0.0,0.0,0.0,710.0,2024-12-20 21:00:00,936.0
9523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-27 20:14:40,SPY_122024P715,300.43,PUT,SPY Dec 20 2024 715 Put,True,2022-05-27 20:14:40,17.333,-1.0,0.0,0.0,0.0,0.0,715.0,2024-12-20 21:00:00,936.0


## 1c. Import data from Yahoo

In [8]:
import pandas_datareader.data as web

all_data = {ticker: web.get_data_yahoo(ticker)       #dictionary comprehension
           for ticker in ["^VIX","SPY"]}


try:
    in_yahoo = pd.DataFrame({ticker: data ['Adj Close']
               for ticker, data in all_data.items()})
except:
    print("Error encountered during copy from Yahoo Finance, should self resolve during market hours. putting [^VIX,SPY] results in error during 315am to 915am ET due to uneven length of columns")

# Calculate EMAs
data = []
in_yahoo0 = in_yahoo.fillna(method="ffill")                 
display(in_yahoo0)

,^VIX,SPY
Date,,
2017-05-30,10.380000,221.303299
2017-05-31,10.410000,221.248352
2017-06-01,9.890000,223.007782
2017-06-02,9.750000,223.750061
2017-06-05,10.070000,223.585098
...,...,...
2022-05-24,29.450001,393.890015
2022-05-25,28.370001,397.369995
2022-05-26,27.500000,405.309998


## 2. Main backtesting code for Verticals

In [95]:
# holding lot for dataframes, run code from here on instead to avoid having to compile code above
in_all = in_all0.copy()
in_yahoo = in_yahoo0.copy()
out = pd.DataFrame([])

### 2b. Normalise 30 min data into daily data

In [96]:
def normalize(in_all):            # normalise 30min data into daily data from SPY files

    in_all.set_index('quoteTimeInLong_ts', inplace=True)     #NEW
    in_all.index = in_all.index.normalize()      #normalise the datetime to dates only
    in_all = in_all[['symbol','mark']]
    in_all = in_all.groupby(['quoteTimeInLong_ts','symbol'])['mark'].mean()      #this dataframe df shows the average mark price of each option for each day
    return in_all

in_all = normalize(in_all)

### 2c. Initialise Parameters for options 

In [97]:
def init_param():
       
    master_list = []

# #     mini test list
#     dte = [18, 21]
#     days_to_close = [7, 9]
#     take_prof = [0.9]
#     take_loss = [1]

#     #medium test list
#     dte = [18,21,24,27,32]
#     days_to_close = [7,9,12,16]
#     take_prof = [0.2,0.4,0.6,0.8,1.0]
#     take_loss = [0.2,0.4,0.6,0.8,1.0]
#     #5*4*3*6 = 360 loops -> 4 hrs
    #5*4*5*9 = 900 loops -> 10 hrs
        
    dte = [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
    days_to_close = [7, 8, 9, 12, 13, 14, 15, 16]                      # skipping days which will result in weekends
    take_prof = [0.5, 0.6, 0.7, 0.8, 0.9, 1]
    take_loss = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
   # 15*8*6*10 = 7200
    master_list.clear()

    for i in range(len(dte)):
        for j in range(len(days_to_close)):
            for k in range(len(take_prof)):
                for l in range(len(take_loss)):
                    master_list.append([dte[i], days_to_close[j], take_prof[k], take_loss[l]])

    master_list = np.array(master_list)        #convert to numpy array
    no_of_iter = len(dte) * len(days_to_close) * len(take_prof) * len(take_loss)    # returns no of iterations for later use
    
    return master_list, no_of_iter, days_to_close

mast_list, no_of_iter, days_to_close = init_param()

In [98]:
def false_func():
    pass

### 2d. Create open date column in out df

In [99]:
def open_date():      # create open_date column
    date = pd.Timestamp(datetime.date.today()).round(freq = 'D')- pd.offsets.Week(weekday=2)
    
    try:
        in_yahoo.loc[date][1]
    
    except:
        print("Key error because you are probably searching for today's SPY price which doesnt exist (yet)!","\nDate you searched for:",date)
        date = date - pd.Timedelta("7 day")
        print("Decreasing days by 7 to try..")
    
    finally:
        list_dates = []
        
        while pd.Timestamp(2022,1,1,0) < date < pd.Timestamp(datetime.date.today()):       # initialising trades every 7 days
                     
            list_dates.append(date)
            date -= pd.Timedelta("7 day")

    # create columns of params
 
    out_param = np.repeat(mast_list, len(list_dates), axis=0)                                    # repeat params over length of list dates
    out_param = pd.DataFrame(out_param)
    out_dates = pd.DataFrame(list_dates)
    out_dates = pd.concat([out_dates] * no_of_iter, axis=0)                            # repeat the open_dates n times, based on the no. of iterations  
    out_dates.reset_index(inplace=True)
    
    out = pd.concat([out_dates, out_param], axis=1) 
    out.columns = ['trade_no','open_date', 'DTE', 'days_to_close', 'take_prof','take_loss']   #rename columns
    out = pd.merge(out, in_yahoo, left_on = ['open_date'], right_on = ['Date'], how='left')        #merge out and in_yahoo together 

    return out

out = open_date()

,index,0,0,1,2,3
0,0,2022-05-25,18.0,7.0,0.5,0.1
1,1,2022-05-18,18.0,7.0,0.5,0.1
2,2,2022-05-11,18.0,7.0,0.5,0.1
3,3,2022-05-04,18.0,7.0,0.5,0.1
4,4,2022-04-27,18.0,7.0,0.5,0.1
...,...,...,...,...,...,...
151195,16,2022-02-02,32.0,16.0,1.0,1.0
151196,17,2022-01-26,32.0,16.0,1.0,1.0
151197,18,2022-01-19,32.0,16.0,1.0,1.0
151198,19,2022-01-12,32.0,16.0,1.0,1.0


### 2e. Determining options to buy/sell based on params

In [100]:
def open_trade(df):
    df['opex_date'] = df.open_date + pd.to_timedelta(df.DTE, unit='d') + pd.offsets.Week(weekday=4)        #changed pd.DateOffset(days = 25) to pd.to_timedelta(df.DTE, unit='d')
    df['opex_date'] = df['opex_date'].astype(str)
    df['opex_date'] = df.opex_date.str[5:7] + df.opex_date.str[8:10] + df.opex_date.str[2:4]
    
    df['spy_upper'] = df['SPY']    
    df['spy_upper'] = df['spy_upper'] - (df['spy_upper'] % 5)                  # round to nearest 5
    df['spy_lower'] = df['spy_upper'] - 25
    
    df['spy_upper'] = df['spy_upper'].astype(int).astype(str)
    df['spy_lower'] = df['spy_lower'].astype(int).astype(str)
    
    df['spy_upper'] = "SPY_" + df['opex_date'] + "P" + df['spy_upper']
    df['spy_lower'] = "SPY_" + df['opex_date'] + "P" + df['spy_lower']
    
    df.drop(['opex_date'], axis=1, inplace=True)
    
    return df

open_trade(out)

,trade_no,open_date,DTE,days_to_close,take_prof,take_loss,^VIX,SPY,spy_upper,spy_lower
0,0,2022-05-25,18.0,7.0,0.5,0.1,28.370001,397.369995,SPY_061722P395,SPY_061722P370
1,1,2022-05-18,18.0,7.0,0.5,0.1,30.959999,391.859985,SPY_061022P390,SPY_061022P365
2,2,2022-05-11,18.0,7.0,0.5,0.1,32.560001,392.750000,SPY_060322P390,SPY_060322P365
3,3,2022-05-04,18.0,7.0,0.5,0.1,25.420000,429.059998,SPY_052722P425,SPY_052722P400
4,4,2022-04-27,18.0,7.0,0.5,0.1,31.600000,417.269989,SPY_052022P415,SPY_052022P390
...,...,...,...,...,...,...,...,...,...,...
151195,16,2022-02-02,32.0,16.0,1.0,1.0,22.090000,455.933594,SPY_031122P455,SPY_031122P430
151196,17,2022-01-26,32.0,16.0,1.0,1.0,31.959999,432.037842,SPY_030422P430,SPY_030422P405
151197,18,2022-01-19,32.0,16.0,1.0,1.0,23.850000,450.350922,SPY_022522P450,SPY_022522P425
151198,19,2022-01-12,32.0,16.0,1.0,1.0,17.620001,469.561249,SPY_021822P465,SPY_021822P440


### 2f. Updating options interim prices via columns spy_upp+ and spy_lower+

In [101]:
def interim_price(out, in_all):    # this function merges the out and in_all dfs together (using merge as a sort of vlookup) and returns the 0 to 16 day SPY option prices as columns.
    
    merged = out.copy()
    
    for days in range(0, 17):
        merged = pd.merge(merged, in_all, 
                          left_on = ['spy_upper', out['open_date'] + datetime.timedelta(days)], 
                          right_on = ['symbol','quoteTimeInLong_ts'], 
                          how ='left')
        merged.rename(columns = {'mark':"spy_upp+" + str(days)}, inplace=True)
    
    for days in range(0, 17):        
        merged = pd.merge(merged, in_all, 
                          left_on = ['spy_lower', out['open_date'] + datetime.timedelta(days)], 
                          right_on = ['symbol','quoteTimeInLong_ts'], 
                          how = 'left')
        merged.rename(columns = {'mark':"spy_lower+" + str(days)}, inplace=True)

    return merged

out = interim_price(out, in_all)

In [102]:
#drop out 1st 2 rows due to incomplete data
out.set_index('trade_no', inplace=True)
out.drop([0, 1], inplace=True)

display(out.head(10))


,quoteTimeInLong_ts,open_date,DTE,days_to_close,take_prof,take_loss,^VIX,SPY,spy_upper,spy_lower,spy_upp+0,spy_upp+1,spy_upp+2,spy_upp+3,spy_upp+4,spy_upp+5,spy_upp+6,spy_upp+7,spy_upp+8,spy_upp+9,spy_upp+10,spy_upp+11,spy_upp+12,spy_upp+13,spy_upp+14,spy_upp+15,spy_upp+16,spy_lower+0,spy_lower+1,spy_lower+2,spy_lower+3,spy_lower+4,spy_lower+5,spy_lower+6,spy_lower+7,spy_lower+8,spy_lower+9,spy_lower+10,spy_lower+11,spy_lower+12,spy_lower+13,spy_lower+14,spy_lower+15,spy_lower+16
trade_no,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,2022-05-11,2022-05-11,18.0,7.0,0.5,0.1,32.560001,392.750000,SPY_060322P390,SPY_060322P365,8.787143,11.882143,6.408293,NaN,NaN,5.657857,3.523571,7.051429,8.888571,9.086190,NaN,NaN,5.404286,7.227857,4.637857,1.684286,0.341707,2.900000,4.145714,1.800244,NaN,NaN,1.363571,0.756429,1.737857,2.275714,2.027381,NaN,NaN,0.940000,1.250000,0.630714,0.229286,0.090244
3,2022-05-04,2022-05-04,18.0,7.0,0.5,0.1,25.420000,429.059998,SPY_052722P425,SPY_052722P400,13.824286,16.206429,18.944524,NaN,NaN,25.995385,27.169286,28.667857,35.492857,25.042439,NaN,NaN,24.811429,19.578571,29.685714,34.800000,36.642381,5.157143,6.400714,7.304048,NaN,NaN,10.873846,11.200714,11.573571,15.675714,8.495122,NaN,NaN,7.717857,4.689286,9.822143,12.705000,13.509524
4,2022-04-27,2022-04-27,18.0,7.0,0.5,0.1,31.600000,417.269989,SPY_052022P415,SPY_052022P390,9.671429,7.765000,11.266905,NaN,NaN,13.258571,9.415000,8.088571,9.904286,11.644762,NaN,NaN,17.395385,18.274286,19.430714,25.720714,15.472439,3.747857,2.972857,4.401190,NaN,NaN,5.160000,3.102143,2.367143,3.053571,3.204524,NaN,NaN,5.391538,5.412143,5.354286,8.162143,2.845854
5,2022-04-20,2022-04-20,18.0,7.0,0.5,0.1,20.320000,444.709991,SPY_051322P440,SPY_051322P415,5.892857,7.034286,16.913333,NaN,NaN,19.348571,21.599286,22.029286,18.343571,25.666429,NaN,NaN,29.715714,23.948571,22.081429,25.399286,29.867619,1.596429,2.013571,6.326429,NaN,NaN,7.385000,8.405000,8.255714,6.379286,9.604524,NaN,NaN,11.557857,7.676429,6.375714,7.965714,9.495952
6,2022-04-13,2022-04-13,18.0,7.0,0.5,0.1,21.820000,443.309998,SPY_050622P440,SPY_050622P415,7.939286,7.896429,NaN,NaN,NaN,8.862143,6.210714,4.767143,5.832857,15.903095,NaN,NaN,18.450000,20.779286,21.305000,17.327857,25.090476,2.457143,2.212143,NaN,NaN,NaN,2.303571,1.385000,0.975000,1.263571,4.863571,NaN,NaN,5.750714,6.530000,6.329286,4.466429,7.291190
7,2022-04-06,2022-04-06,18.0,7.0,0.5,0.1,22.100000,446.519989,SPY_042922P445,SPY_042922P420,8.194167,7.410000,5.994286,NaN,NaN,9.038571,8.976154,8.387143,8.468571,NaN,NaN,NaN,9.785714,6.425000,4.653571,5.903571,18.676905,2.684167,2.268571,1.625714,NaN,NaN,2.532143,2.388462,1.948571,1.650000,NaN,NaN,NaN,1.641429,0.830000,0.511429,0.657143,3.762143
8,2022-03-30,2022-03-30,18.0,7.0,0.5,0.1,19.330000,458.700012,SPY_042222P455,SPY_042222P430,NaN,6.390000,8.388750,NaN,NaN,4.950000,6.820833,11.641667,10.811429,8.887143,NaN,NaN,13.824286,14.074615,13.795000,14.714286,NaN,NaN,1.760000,1.932500,NaN,NaN,0.890000,1.363333,2.824167,2.355714,1.557857,NaN,NaN,2.637857,2.400769,1.895000,1.502857,NaN
9,2022-03-23,2022-03-23,18.0,7.0,0.5,0.1,23.570000,443.799988,SPY_041522P440,SPY_041522P415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2022-03-16,2022-03-16,18.0,7.0,0.5,0.1,26.670000,434.270874,SPY_040822P430,SPY_040822P405,10.351429,7.409286,5.317143,NaN,NaN,4.280714,2.815000,3.137143,2.504286,1.412381,NaN,NaN,1.112500,NaN,NaN,0.340000,0.242500,3.911429,2.615714,1.847143,NaN,NaN,1.341429,0.878571,0.905714,0.695714,0.393810,NaN,NaN,0.281667,NaN,NaN,0.125000,0.078750


In [103]:
# fill missing values with interpolate()

out_low = out.filter(like= 'spy_lower+', axis=1).copy()            #filter out spy_lower columns and fill na
out_low.interpolate(method='bfill', inplace=True, axis=1)

out_upp = out.filter(like= 'spy_upp+', axis=1).copy()             #filter out spy_upp columns and fill na
out_upp.interpolate(method='bfill', inplace=True, axis=1)

for days in range(0,17):                                                 #copy back the columns into main out df
    out['spy_upp+' + str(days)] = out_upp['spy_upp+' + str(days)]
    out['spy_lower+' + str(days)] = out_low['spy_lower+' + str(days)]
display(out)

,quoteTimeInLong_ts,open_date,DTE,days_to_close,take_prof,take_loss,^VIX,SPY,spy_upper,spy_lower,spy_upp+0,spy_upp+1,spy_upp+2,spy_upp+3,spy_upp+4,spy_upp+5,spy_upp+6,spy_upp+7,spy_upp+8,spy_upp+9,spy_upp+10,spy_upp+11,spy_upp+12,spy_upp+13,spy_upp+14,spy_upp+15,spy_upp+16,spy_lower+0,spy_lower+1,spy_lower+2,spy_lower+3,spy_lower+4,spy_lower+5,spy_lower+6,spy_lower+7,spy_lower+8,spy_lower+9,spy_lower+10,spy_lower+11,spy_lower+12,spy_lower+13,spy_lower+14,spy_lower+15,spy_lower+16
trade_no,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,2022-05-11,2022-05-11,18.0,7.0,0.5,0.1,32.560001,392.750000,SPY_060322P390,SPY_060322P365,8.787143,11.882143,6.408293,5.657857,5.657857,5.657857,3.523571,7.051429,8.888571,9.086190,5.404286,5.404286,5.404286,7.227857,4.637857,1.684286,0.341707,2.900000,4.145714,1.800244,1.363571,1.363571,1.363571,0.756429,1.737857,2.275714,2.027381,0.940000,0.940000,0.940000,1.250000,0.630714,0.229286,0.090244
3,2022-05-04,2022-05-04,18.0,7.0,0.5,0.1,25.420000,429.059998,SPY_052722P425,SPY_052722P400,13.824286,16.206429,18.944524,25.995385,25.995385,25.995385,27.169286,28.667857,35.492857,25.042439,24.811429,24.811429,24.811429,19.578571,29.685714,34.800000,36.642381,5.157143,6.400714,7.304048,10.873846,10.873846,10.873846,11.200714,11.573571,15.675714,8.495122,7.717857,7.717857,7.717857,4.689286,9.822143,12.705000,13.509524
4,2022-04-27,2022-04-27,18.0,7.0,0.5,0.1,31.600000,417.269989,SPY_052022P415,SPY_052022P390,9.671429,7.765000,11.266905,13.258571,13.258571,13.258571,9.415000,8.088571,9.904286,11.644762,17.395385,17.395385,17.395385,18.274286,19.430714,25.720714,15.472439,3.747857,2.972857,4.401190,5.160000,5.160000,5.160000,3.102143,2.367143,3.053571,3.204524,5.391538,5.391538,5.391538,5.412143,5.354286,8.162143,2.845854
5,2022-04-20,2022-04-20,18.0,7.0,0.5,0.1,20.320000,444.709991,SPY_051322P440,SPY_051322P415,5.892857,7.034286,16.913333,19.348571,19.348571,19.348571,21.599286,22.029286,18.343571,25.666429,29.715714,29.715714,29.715714,23.948571,22.081429,25.399286,29.867619,1.596429,2.013571,6.326429,7.385000,7.385000,7.385000,8.405000,8.255714,6.379286,9.604524,11.557857,11.557857,11.557857,7.676429,6.375714,7.965714,9.495952
6,2022-04-13,2022-04-13,18.0,7.0,0.5,0.1,21.820000,443.309998,SPY_050622P440,SPY_050622P415,7.939286,7.896429,8.862143,8.862143,8.862143,8.862143,6.210714,4.767143,5.832857,15.903095,18.450000,18.450000,18.450000,20.779286,21.305000,17.327857,25.090476,2.457143,2.212143,2.303571,2.303571,2.303571,2.303571,1.385000,0.975000,1.263571,4.863571,5.750714,5.750714,5.750714,6.530000,6.329286,4.466429,7.291190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,2022-02-02,2022-02-02,32.0,16.0,1.0,1.0,22.090000,455.933594,SPY_031122P455,SPY_031122P430,9.790000,15.820000,16.580000,13.180000,13.180000,13.180000,10.750000,7.750000,12.420000,18.720000,19.560000,19.560000,19.560000,14.360000,13.258000,17.053750,NaN,3.790000,6.080000,5.050000,5.890000,5.890000,5.890000,4.110000,2.850000,5.090000,8.070000,8.140000,8.140000,8.140000,5.190000,4.667000,6.407500,NaN
17,2022-01-26,2022-01-26,32.0,16.0,1.0,1.0,31.959999,432.037842,SPY_030422P430,SPY_030422P405,12.560000,13.650000,8.540000,5.420000,5.420000,5.420000,3.880000,2.880000,5.000000,4.800000,4.340000,4.340000,4.340000,3.150000,1.940000,3.960000,6.500000,6.450000,6.280000,4.250000,2.520000,2.520000,2.520000,1.620000,1.300000,2.580000,1.760000,1.610000,1.610000,1.610000,1.040000,0.660000,1.340000,2.500000
18,2022-01-19,2022-01-19,32.0,16.0,1.0,1.0,23.850000,450.350922,SPY_022522P450,SPY_022522P425,10.980000,13.420000,17.830000,17.440000,17.440000,17.440000,19.950000,22.300000,23.510000,16.300000,9.990000,9.990000,9.990000,6.940000,6.880000,9.110000,8.840000,4.610000,6.000000,8.410000,8.180000,8.180000,8.180000,10.550000,10.270000,10.520000,6.650000,3.620000,3.620000,3.620000,2.340000,1.860000,3.780000,2.910000


### 2g. Calculating P/L for trades with normal expiry (i.e. expire by days_to_close)

In [104]:
# calculate p/l for the rest of the trades which are not triggered by take_prof or take_loss
out['credit_0'] = out['spy_upp+0'] - out['spy_lower+0'] 


In [105]:
for days in days_to_close:
#     print(days)
    out['norm_pl+'+str(days)] = np.where(out['days_to_close']==days, 
                         out['credit_0'] - (out['spy_upp+'+str(days)] - out['spy_lower+'+str(days)]),                    
                         false_func())

for i in range(len(days_to_close)-1):                                               # merging the data from different days into a single column, at last element of days_to_close
      out['norm_pl+'+str(days_to_close[i+1])] = out['norm_pl+'+str(days_to_close[i])].combine_first(out['norm_pl+'+str(days_to_close[i+1])])
      
out['norm_pl'] = out['norm_pl+'+str(days_to_close[-1])]
    

### 2h. Determining trades which need early exit (i.e. create takeprof_day and takeloss_day)

In [106]:
#creating takeprof_day+X and takeprof_day+Y to compute the percentages
for days in range(0,17):
    credit_0 = out['spy_upp+0'] - out['spy_lower+0']         # credit on day 0
    credit_day = out['spy_upp+' + str(days)] - out['spy_lower+'+ str(days)]       # credit of that day
    out['takeprof_day+' + str(days)] = ((credit_0 - credit_day) / credit_0 > out['take_prof'])           # if the take prof ratio that day is larger than 'take prof' column, return True
        
    max_loss = 25 - credit_0
    out['takeloss_day+' + str(days)] = ((credit_0 - credit_day) / max_loss) < (out['take_loss'] * -1)       # if the take loss ratio that day is larger than 'take loss' column, return True
    
display(out)    

,quoteTimeInLong_ts,open_date,DTE,days_to_close,take_prof,take_loss,^VIX,SPY,spy_upper,spy_lower,spy_upp+0,spy_upp+1,spy_upp+2,spy_upp+3,spy_upp+4,spy_upp+5,spy_upp+6,spy_upp+7,spy_upp+8,spy_upp+9,spy_upp+10,spy_upp+11,spy_upp+12,spy_upp+13,spy_upp+14,spy_upp+15,spy_upp+16,spy_lower+0,spy_lower+1,spy_lower+2,spy_lower+3,spy_lower+4,spy_lower+5,spy_lower+6,spy_lower+7,spy_lower+8,spy_lower+9,spy_lower+10,spy_lower+11,spy_lower+12,spy_lower+13,spy_lower+14,spy_lower+15,spy_lower+16,credit_0,norm_pl+7,norm_pl+8,norm_pl+9,norm_pl+12,norm_pl+13,norm_pl+14,norm_pl+15,norm_pl+16,norm_pl,takeprof_day+0,takeloss_day+0,takeprof_day+1,takeloss_day+1,takeprof_day+2,takeloss_day+2,takeprof_day+3,takeloss_day+3,takeprof_day+4,takeloss_day+4,takeprof_day+5,takeloss_day+5,takeprof_day+6,takeloss_day+6,takeprof_day+7,takeloss_day+7,takeprof_day+8,takeloss_day+8,takeprof_day+9,takeloss_day+9,takeprof_day+10,takeloss_day+10,takeprof_day+11,takeloss_day+11,takeprof_day+12,takeloss_day+12,takeprof_day+13,takeloss_day+13,takeprof_day+14,takeloss_day+14,takeprof_day+15,takeloss_day+15,takeprof_day+16,takeloss_day+16
trade_no,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,2022-05-11,2022-05-11,18.0,7.0,0.5,0.1,32.560001,392.750000,SPY_060322P390,SPY_060322P365,8.787143,11.882143,6.408293,5.657857,5.657857,5.657857,3.523571,7.051429,8.888571,9.086190,5.404286,5.404286,5.404286,7.227857,4.637857,1.684286,0.341707,2.900000,4.145714,1.800244,1.363571,1.363571,1.363571,0.756429,1.737857,2.275714,2.027381,0.940000,0.940000,0.940000,1.250000,0.630714,0.229286,0.090244,5.887143,0.573571,0.573571,0.573571,0.573571,0.573571,0.573571,0.573571,0.573571,0.573571,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False
3,2022-05-04,2022-05-04,18.0,7.0,0.5,0.1,25.420000,429.059998,SPY_052722P425,SPY_052722P400,13.824286,16.206429,18.944524,25.995385,25.995385,25.995385,27.169286,28.667857,35.492857,25.042439,24.811429,24.811429,24.811429,19.578571,29.685714,34.800000,36.642381,5.157143,6.400714,7.304048,10.873846,10.873846,10.873846,11.200714,11.573571,15.675714,8.495122,7.717857,7.717857,7.717857,4.689286,9.822143,12.705000,13.509524,8.667143,-8.427143,-8.427143,-8.427143,-8.427143,-8.427143,-8.427143,-8.427143,-8.427143,-8.427143,False,False,False,False,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True
4,2022-04-27,2022-04-27,18.0,7.0,0.5,0.1,31.600000,417.269989,SPY_052022P415,SPY_052022P390,9.671429,7.765000,11.266905,13.258571,13.258571,13.258571,9.415000,8.088571,9.904286,11.644762,17.395385,17.395385,17.395385,18.274286,19.430714,25.720714,15.472439,3.747857,2.972857,4.401190,5.160000,5.160000,5.160000,3.102143,2.367143,3.053571,3.204524,5.391538,5.391538,5.391538,5.412143,5.354286,8.162143,2.845854,5.923571,0.202143,0.202143,0.202143,0.202143,0.202143,0.202143,0.202143,0.202143,0.202143,False,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True
5,2022-04-20,2022-04-20,18.0,7.0,0.5,0.1,20.320000,444.709991,SPY_051322P440,SPY_051322P415,5.892857,7.034286,16.913333,19.348571,19.348571,19.348571,21.599286,22.029286,18.343571,25.666429,29.715714,29.715714,29.715714,23.948571,22.081429,25.399286,29.867619,1.596429,2.013571,6.326429,7.385000,7.385000,7.385000,8.405000,8.255714,6.379286,9.604524,11.557857,11.557857,11.557857,7.676429,6.375714,7.965714,9.495952,4.296429,-9.477143,-9.477143,-9.477143,-9.477143,-9.477143,-9.477143,-9.477143,-9.477143,-9.477143,False,False,False,False,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False

### 2i. Create actual profit columns (actual_prof) which computes actual profit when taken early

In [107]:
# creates actual profit columns (by days) which computes actual profit when profit is taken early

out['credit_0'] = out['spy_upp+0'] - out['spy_lower+0']

for days in range(16,-1,-1):

    out['credit_day'] = out['spy_upp+' + str(days)] - out['spy_lower+'+ str(days)] 
    out['early_prof+'+ str(days)] = np.where(out['takeprof_day+'+str(days)]==True, 
                         out['credit_0'] - out['credit_day'], 
                         false_func())
out = out.copy()                                                 #to remove fragmented df warning

### 2j. Create actual loss columns (actual_loss) which computes actual profit when taken early

In [108]:
# creates actual loss columns (by days) which computes actual loss when loss is taken early
# working well

for days in range(16,-1,-1):

    out['credit_day_l'] = out['spy_upp+' + str(days)] - out['spy_lower+'+ str(days)] 
    out['early_loss+'+ str(days)] = np.where(out['takeloss_day+'+str(days)]==True, 
                                              out['credit_0'] - out['credit_day_l'] , 
                                              false_func())


### 2k. Combine actual prof/loss into actual_pl

In [109]:
#combine actual_prof and actual_loss by day
for days in range(0,17):
    out['early_pl+'+ str(days)] = out['early_loss+'+ str(days)].combine_first(out['early_prof+'+ str(days)])


### 2l. Combine all p/l into final_pl

In [110]:
#combine actual_pl into 1 column only

for days in range(0,16):
    
    days_plus_1 = days + 1
    out['early_pl+'+ str(days_plus_1)] = out['early_pl+'+ str(days)].combine_first(out['early_pl+'+ str(days_plus_1)])


In [111]:
def check_early_exit(df):
    
    early_exit = False
    
    for days in range(0, int(df['days_to_close'])+1):
        if (df['takeprof_day+'+str(days)] == True) or (df['takeloss_day+'+str(days)] == True):
            early_exit = True
                               
    if early_exit == True:
        return df['early_pl+'+str(days)]    #, df['remarks']   # update remarks - early exit on day X
    else:
        return df['norm_pl']                #, df['remarks']               # update remarks - normal exit 
        
out['final_pl'] = out.apply(check_early_exit, axis=1)

In [112]:
out['max_loss'] = 25 - (out['spy_upp+0'] - out['spy_lower+0'])
out['final_pl%'] = out['final_pl'] / out['max_loss'] 

In [113]:
# create new column combi_no to groupby later

reps = out.index.nunique()
count = range(len(out.index) // int(reps))
combi_no = np.repeat(count, reps)
out['combi_no'] = combi_no

# calculate mean for each combi and manipulate to correct format
combi = out.groupby('combi_no')['final_pl%'].mean()
series = pd.Series(np.repeat(combi, reps))
series = series.rename('mean_pl%')
series = series.reset_index()
series = series['mean_pl%']
series = series.copy()

out = out.reset_index()
out = pd.concat([out,series], axis=1)             # copy to out df.

out['last5MA'] = out['final_pl%'].rolling(window=5).mean()


In [131]:
out.to_csv('out116.csv')

KeyboardInterrupt: 

In [166]:
# lowest mean_pl%
out2 = out[['trade_no','open_date','DTE','days_to_close','take_prof','take_loss','mean_pl%','combi_no','last5MA','final_pl%']]
out2.set_index('trade_no', inplace=True)

# Some performance tracking

## Check last 5 MA with best performance 

In [177]:
display(out2.loc[6].sort_values(by='last5MA').head(3))
display(out2.loc[7].sort_values(by='last5MA').head(3))
display(out2.loc[8].sort_values(by='last5MA').head(3))

,open_date,DTE,days_to_close,take_prof,take_loss,mean_pl%,combi_no,last5MA,final_pl%
trade_no,,,,,,,,,
6,2022-04-13,22.0,16.0,0.5,0.9,-0.275335,2348,-0.496267,-0.63107
6,2022-04-13,22.0,16.0,0.5,1.0,-0.275335,2349,-0.496267,-0.63107
6,2022-04-13,21.0,16.0,0.5,1.0,-0.275335,1869,-0.496267,-0.63107


,open_date,DTE,days_to_close,take_prof,take_loss,mean_pl%,combi_no,last5MA,final_pl%
trade_no,,,,,,,,,
7,2022-04-06,22.0,16.0,0.8,0.9,-0.252790,2378,-0.625424,-0.482543
7,2022-04-06,22.0,16.0,0.7,1.0,-0.262283,2369,-0.625424,-0.482543
7,2022-04-06,22.0,16.0,0.5,1.0,-0.275335,2349,-0.625424,-0.482543


,open_date,DTE,days_to_close,take_prof,take_loss,mean_pl%,combi_no,last5MA,final_pl%
trade_no,,,,,,,,,
8,2022-03-30,22.0,16.0,0.6,0.4,-0.218335,2353,-0.442032,-0.421278
8,2022-03-30,22.0,16.0,0.7,0.4,-0.212796,2363,-0.442032,-0.421278
8,2022-03-30,22.0,16.0,0.9,0.4,-0.197448,2383,-0.442032,-0.421278


## Checks the top mean_pl for all trades from Jan 2022

In [174]:
out2.groupby(by='mean_pl%',sort='True').mean()

,DTE,days_to_close,take_prof,take_loss,combi_no,last5MA,final_pl%
mean_pl%,,,,,,,
-0.283029,20.0,16.0,0.50,0.80,1387.0,-0.342018,-0.283029
-0.276476,20.0,16.0,0.50,0.70,1386.0,-0.332623,-0.276476
-0.275863,20.0,16.0,0.60,0.80,1397.0,-0.332486,-0.275863
-0.275335,20.0,16.0,0.50,0.95,1388.5,-0.349943,-0.275335
-0.269978,20.0,16.0,0.70,0.80,1407.0,-0.329366,-0.269978
...,...,...,...,...,...,...,...
-0.051981,31.0,7.0,0.85,0.10,6275.0,-0.074948,-0.051981
-0.049646,20.0,7.0,0.60,0.10,970.0,-0.066441,-0.049646
-0.047384,20.0,7.0,0.70,0.10,980.0,-0.063433,-0.047384


From above, we can tell that the worst strategy is using DTE=20, days_to_close=16, take_prof=0.5, and take_loss=0.8. The YTD loss is 28.3%.
On the flipside, we would have generated the best profit if the reverse strategy was traded - i.e. buy a put ATM, sell a put 25 points below.
This is so far a viable strategy, however, close monitoring is required to see if the strategy continues to perform well in the future due to fast changing market conditions.